<a href="https://colab.research.google.com/github/ab17254/ce802/blob/main/CE802_P3_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving CE802_P3_Data.csv to CE802_P3_Data.csv
Saving CE802_P3_Test.csv to CE802_P3_Test.csv
User uploaded file "CE802_P3_Data.csv" with length 150646 bytes
User uploaded file "CE802_P3_Test.csv" with length 143732 bytes


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor

from sklearn import svm

sns.set_style('whitegrid')

In [3]:
data = pd.read_csv('/content/CE802_P3_Data.csv')
test = pd.read_csv('/content/CE802_P3_Test.csv')

In [4]:
data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target
0,5.82,Low,15.92,-21480.14,2.54,8,-54.21,11.19,-148.74,5.76,-1567.27,61.06,5.86,3861.39,Rest,9,1342.36
1,14.90,Medium,1.06,-30658.72,-2.50,4,-89.22,4.29,-98.80,-7.95,-1440.25,122.98,78.50,3546.00,Rest,9,204.91
2,8.64,Low,1.75,-36063.70,-2.36,8,70.68,-0.33,-141.74,2.34,-1213.40,130.64,4.12,6068.73,UK,6,0.00
3,2.16,Very high,3.95,-39859.52,-6.96,12,104.70,14.07,-88.87,32.10,-773.05,104.01,0.34,4749.24,UK,3,0.00
4,14.36,Low,2.36,-40725.00,3.60,12,-65.61,7.47,-15.16,1.59,-906.33,69.84,0.06,2969.61,Rest,15,0.00


In [5]:
test.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target
0,3.06,High,1.32,-50743.74,-1.08,2,10.11,1.74,-67.98,16.17,-1257.99,29.75,6.96,4750.68,Europe,12,NaN
1,9.98,Medium,2.51,-57574.32,-0.75,12,-126.69,9.84,-150.96,0.66,-1257.20,104.60,0.02,3623.19,UK,6,NaN
2,2.30,Very low,3.43,-31777.88,-0.03,8,-21.75,10.62,-124.71,11.94,-345.02,154.47,315.58,4274.67,Europe,9,NaN
3,2.42,Very high,6.67,-42583.36,7.02,8,-185.70,-5.40,-93.20,6.60,-1276.61,40.70,6.40,3892.74,UK,3,NaN
4,2.24,Very high,3.23,-39844.64,-2.44,4,28.41,14.07,-68.80,1.89,-1199.55,85.06,0.20,4624.62,UK,15,NaN


In [6]:
def clean_data(data):
  data.replace({'F2':{'Very low': 0, 'Low':1, 'Medium':2, 'High':3, 'Very high':4}}, inplace=True)
  clean_data = pd.get_dummies(data, columns=['F15'])
  return(clean_data)

In [7]:
data = clean_data(data)
test_data = clean_data(test)

In [8]:
x = data.loc[:, data.columns != 'Target'].to_numpy()
y = data.loc[:, data.columns == 'Target'].to_numpy()
print(x.shape, y.shape)

(1500, 19) (1500, 1)


In [9]:
x_test = test_data.loc[:, test_data.columns != 'Target'].to_numpy()
print(x_test.shape)

(1500, 19)


In [10]:
k_fold = KFold(n_splits=5)
for train_indices, test_indices in k_fold.split(x):
    ss = StandardScaler()
    ss.fit(x[train_indices, :])
    x_train = ss.transform(x[train_indices, :])
    x_test = ss.transform(x[test_indices, :])
    y_train = y[train_indices]
    y_test = y[test_indices]

## a) Investigate the performance of a number of machine learning procedures on this dataset.

### Linear Regression

In [11]:
model_lin_reg = LinearRegression()
model_lin_reg.fit(x_train, y_train.ravel())
lin_reg_pred = model_lin_reg.predict(x_test)
print('Mean Squared Error: %.2f'% mean_squared_error(y_test, lin_reg_pred))
print('Coefficient of Determination: %.2f'% r2_score(y_test, lin_reg_pred))

Coefficients:
  [ 3.81075612e+02  2.22193716e+02  4.02114574e+02  1.67482644e+01
 -3.14368912e+00 -2.33586382e+02 -8.09033621e+00  2.07370104e+01
 -4.30880243e+02  2.17854706e+00  2.92709922e+02  2.02291654e+01
  5.31196188e+00 -2.53927950e+02  2.38125524e+02  1.68021897e+16
  1.68397781e+16  1.73320386e+16  1.63243259e+16]
Mean Squared Error: 273213.37
Coefficient of Determination: 0.79


In [12]:
'''
plt.figure(figsize=(15,15))
plt.scatter(x_test[:,0], y_test, color='black')
plt.plot(x_test[:,0], lin_reg_pred, color='blue', linewidth=3)
plt.xticks(())
plt.yticks(())

plt.show()
'''

"\nplt.figure(figsize=(15,15))\nplt.scatter(x_test[:,0], y_test, color='black')\nplt.plot(x_test[:,0], lin_reg_pred, color='blue', linewidth=3)\nplt.xticks(())\nplt.yticks(())\n\nplt.show()\n"

### Support Vector regression

In [13]:
model_svr = svm.SVR()
model_svr.fit(x_train, y_train.ravel())
svr_pred = model_svr.predict(x_test)
print('Mean Squared Error: %.2f'% mean_squared_error(y_test, svr_pred))
print('Coefficient of Determination: %.2f'% r2_score(y_test, svr_pred))

Mean Squared Error: 1520740.30
Coefficient of Determination: -0.20


#### Tuned Support Vector regression

In [17]:
svr_param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001],
              'kernel': ['linear', 'rbf']}
svr_cv = GridSearchCV(model_svr, svr_param_grid, refit=True)
svr_cv.fit(x_train, y_train.ravel())
svr_cv_pred = svr_cv.predict(x_test)
print('Mean Squared Error: %.2f'% mean_squared_error(y_test, svr_cv_pred))
print('Coefficient of Determination: %.2f'% r2_score(y_test, svr_cv_pred))
print(svr_cv.best_params_)

Mean Squared Error: 160644.89
Coefficient of Determination: 0.87
{'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}


### Gradient Boosting Regression

In [15]:
model_grad = GradientBoostingRegressor()
model_grad.fit(x_train, y_train.ravel())
grad_pred = model_grad.predict(x_test)
print('Mean Squared Error: %.2f'% mean_squared_error(y_test, grad_pred))
print('Coefficient of Determination: %.2f'% r2_score(y_test, grad_pred))

Mean Squared Error: 194271.69
Coefficient of Determination: 0.85


#### Tuned Gradient Boosting regression

In [16]:
grad_param_grid = {'n_estimators':[50,100,200,300,400]}
grad_cv = GridSearchCV(model_grad, grad_param_grid, scoring='neg_mean_squared_error', refit=True)
grad_cv.fit(x_train, y_train.ravel())
grad_cv_pred = grad_cv.predict(x_test)
print('Mean Squared Error: %.2f'% mean_squared_error(y_test, grad_cv_pred))
print('Coefficient of Determination: %.2f'% r2_score(y_test, grad_cv_pred))
print(grad_cv.best_params_)

Mean Squared Error: 179930.58
Coefficient of Determination: 0.86
